# maimai 谱面难度预测

## 1. 导入所需库

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 2. 加载数据

### 2.1 加载并解析 maidata.txt

我们使用外部工具 `SimaiSerializerFromMajdataEdit.exe` 来将 `maidata.txt` 格式的谱面文件解析并序列化为 JSON 文件。

**使用方法:**

在终端中执行以下命令，它会将 `data\maichart-converts` 目录下的所有谱面处理并输出到 `data\serialized` 目录。

```bash
src\serializer\src\bin\Release\net8.0\SimaiSerializerFromMajdataEdit.exe data\maichart-converts data\serialized
```

该工具的通用命令格式为： `SimaiSerializerFromMajdataEdit.exe <输入文件或目录> <输出目录>`

执行完毕后，我们就可以在下一步中加载这些生成的 JSON 文件。


In [ ]:
import os
import subprocess

# 路径定义
exe_path = r'src\serializer\src\bin\Release\net8.0\SimaiSerializerFromMajdataEdit.exe'
input_dir = r'data\maichart-converts'
output_dir = r'data\serialized'

# 构建并显示命令
command = f'"{exe_path}" "{input_dir}" "{output_dir}"'
print(f"即将执行的命令: {command}")

# 用户确认
if input("请确认是否要执行此命令 (y/n): ").strip().lower() != 'y':
    print("操作已取消")
    exit()

try:
    # 执行命令
    print("开始执行命令...")
    result = subprocess.run(command, shell=True, check=True, 
                          stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    print("命令执行成功！")
    print("输出结果:")
    print(result.stdout)
    
except subprocess.CalledProcessError as e:
    print("错误信息:", e.stderr)
except Exception as e:
    print(f"发生异常: {str(e)}")

### 2.2 加载谱面标签数据
请将你的谱面标签数据（如 CSV 文件）放置在 `data` 目录下，并在此处加载。
标签数据应至少包含 `song_id` 和 `difficulty_constant` 两列。

In [ ]:
# label_df = pd.read_csv('../data/your_labels.csv')
# print(label_df.head())

## 3. 特征工程

在这一步，我们将从解析后的谱面数据中提取特征。特征可以包括：

- Note 总数
- BPM 变化
- 特定 Note 类型（Tap, Hold, Slide）的数量
- Note 密度
- ...等等

In [ ]:
def feature_engineering(chart_data):
    features = []
    for chart in chart_data:
        # 在这里实现你的特征提取逻辑
        feature_vector = {
            'song_id': chart.get('song_id'), # 假设解析后的数据中有 song_id
            'note_count': len(chart.get('notes', [])), # 示例特征
            # 添加更多特征...
        }
        features.append(feature_vector)
    return pd.DataFrame(features)

# # 加载之前保存的 JSON 数据
# with open('../data/chart_data.json', 'r', encoding='utf-8') as f:
#     chart_data_json = json.load(f)
#
# feature_df = feature_engineering(chart_data_json)
# print(feature_df.head())

## 4. 数据预处理与模型构建

In [ ]:
# # 合并特征和标签
# full_df = pd.merge(feature_df, label_df, on='song_id')
#
# # 分离特征和目标变量
# X = full_df.drop(['song_id', 'difficulty_constant'], axis=1).values
# y = full_df['difficulty_constant'].values
#
# # 数据标准化
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
#
# # 划分训练集和测试集
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
#
# # 转换为 PyTorch Tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

### 4.1 定义模型

In [ ]:
class DifficultyPredictor(nn.Module):
    def __init__(self, input_features):
        super(DifficultyPredictor, self).__init__()
        self.layer1 = nn.Linear(input_features, 128)
        self.layer2 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

# model = DifficultyPredictor(X_train_tensor.shape[1])

## 5. 模型训练

In [ ]:
# # 定义损失函数和优化器
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#
# # 训练循环
# epochs = 100
# for epoch in range(epochs):
#     model.train()
#     optimizer.zero_grad()
#     outputs = model(X_train_tensor)
#     loss = criterion(outputs, y_train_tensor)
#     loss.backward()
#     optimizer.step()
#
#     if (epoch+1) % 10 == 0:
#         print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

## 6. 模型评估

In [ ]:
# model.eval()
# with torch.no_grad():
#     predictions = model(X_test_tensor)
#     test_loss = criterion(predictions, y_test_tensor)
#     print(f'Test Loss: {test_loss.item():.4f}')
#
# # 可以在这里添加更详细的评估指标，例如 MAE, R^2 等

## 7. 结果分析与迭代

分析模型的预测结果，与真实定数进行比较。

思考以下问题：
- 模型的误差主要来自哪些谱面？
- 是否有必要调整特征工程的方案？
- 是否需要更复杂的模型结构？

根据分析结果，回到前面的步骤进行迭代优化。